In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import cvxpy as cp
import gurobipy

from src.problem import Problem
from pytorch.models import FFNet

In [2]:
dh_s = 120*60
initial_storage_MW = 10000.0
max_discharges = 3.0

N = 12
Ts = []
for ii in range(0,288, int(288/N)):
    Ts.append(ii)

In [3]:
supply_df = pd.read_csv('data/CAISO-supply-20210520.csv')
demand_df = pd.read_csv('data/CAISO-demand-20210510.csv')
renewables_df = pd.read_csv('data/CAISO-renewables-20210520.csv')

natural_gas = supply_df.iloc[1].values[1:-1][Ts]
large_hydro = supply_df.iloc[2].values[1:-1][Ts]
imports = supply_df.iloc[3].values[1:-1][Ts]
batteries = supply_df.iloc[4].values[1:-1][Ts]
nuclear = supply_df.iloc[5].values[1:-1][Ts]
coal = supply_df.iloc[6].values[1:-1][Ts]
other = supply_df.iloc[7].values[1:-1][Ts]

solar = renewables_df.iloc[0].values[1:][Ts]
wind = renewables_df.iloc[1].values[1:][Ts]
geothermal = renewables_df.iloc[2].values[1:][Ts]
biomass = renewables_df.iloc[3].values[1:][Ts]
biogas = renewables_df.iloc[4].values[1:][Ts]
small_hydro = renewables_df.iloc[5].values[1:][Ts]

renewables = supply_df.iloc[0].values[1:-1][Ts]
non_renewables = natural_gas + large_hydro + imports + batteries + nuclear + coal + other

n_sources = 7
supply = np.zeros((n_sources, len(Ts)))
supply[0,:] = solar
supply[1] = wind
supply[2] = geothermal
supply[3] = biomass
supply[4] = biogas
supply[5] = small_hydro
supply[6] = non_renewables

day_ahead_forecast = demand_df.iloc[0].values[1:-1][Ts].astype(float)
hour_ahead_forecast = demand_df.iloc[1].values[1:-1][Ts].astype(float)
# demand = demand_df.iloc[2].values[1:-1]

In [4]:
params = {}
params['initial_storage_MW'] = initial_storage_MW
params['max_discharges'] = max_discharges
params['supply'] = np.maximum(supply, 0.0)
params['hour_ahead_forecast'] = hour_ahead_forecast

In [5]:
# prob.solve_bin_problem(params)
N = 12
n_sources = 7
prob = Problem(N, n_sources)

In [6]:
prob_success, cost, solve_time, optvals = prob.solve_bin_problem(params)

Academic license - for non-commercial use only - expires 2021-07-20
Using license file /Users/acauligi/gurobi.lic


In [7]:
y_opt = optvals[0]
prob.solve_coco_problem(params, y_opt)

(True, 277194.0, 0.0037310123443603516)